# Ocean Carrier Alliances - Data Analysis for USDA Report

This notebook analyses data on maritime container shipping to inspect the impacts of strategic alliances between ocean freight carriers (Ocean Carrier Alliances, OCAs) on the containerized agricultural export market in the US. Data are pre-processed in the oca_data_prep notebook; see the [project repo](https://github.com/epistemetrica/Ocean-Carrier-Alliances-Project) for full details. 

In [1]:
#preliminaries
import numpy as np
import pandas as pd
import polars as pl
import plotly_express as px
import statsmodels.api as sm

#enable string cache for polars categoricals
pl.enable_string_cache()
#display settings
pd.set_option('display.max_columns', None)

In [4]:
#load data
main_lf = (
    #load parquets 
    pl.scan_parquet('../data/main/*.parquet')
    #choose only exports
    .filter(pl.col('direction')=='export')
    #drop unneeded columns
    .drop('origin_territory', 'origin_region', 'direction', 
          'carrier_name', 'carrier_scac')
    #rename carrier cols
    .rename({'unified_carrier_name':'carrier_name',
             'unified_carrier_scac':'scac'})
)

In [5]:
main_lf.describe()

statistic,teus,arrival_port_code,arrival_port_name,departure_port_code,departure_port_name,coast_region,hs_code,vessel_name,voyage_number,vessel_id,bol_id,year,month,lane_id,dest_territory,dest_region,lane_name,carrier_name,scac,vessel_owner,primary_cargo,shared_teus,date,alliance,alliance_member,pc_alliance,cargo_source,drewery_lane,dist,rate_40,rate_20,vessel_capacity
str,f64,str,str,str,str,str,str,str,str,f64,str,f64,str,str,str,str,str,str,str,str,f64,f64,str,str,f64,str,str,str,f64,f64,f64,f64
"""count""",6.3737317e7,"""63737317""","""63737317""","""63737317""","""63737317""","""63737173""","""63735980""","""63737317""","""59354831""",6.3737317e7,"""63737316""",6.3737317e7,"""63737317""","""63737317""","""63734881""","""63734881""","""63737317""","""63696714""","""63737317""","""63737317""",6.3737317e7,6.3737317e7,"""63737242""","""63737317""",6.3737317e7,"""63737317""","""63737317""","""63734875""",6.3734875e7,3.5720688e7,3.5720688e7,5.9237959e7
"""null_count""",0.0,"""0""","""0""","""0""","""0""","""144""","""1337""","""0""","""4382486""",0.0,"""1""",0.0,"""0""","""0""","""2436""","""2436""","""0""","""40603""","""0""","""0""",0.0,0.0,"""75""","""0""",0.0,"""0""","""0""","""2442""",2442.0,2.8016629e7,2.8016629e7,4.499358e6
"""mean""",3.220729,null,null,null,null,null,null,null,null,9.2319e6,null,2015.465141,null,null,null,null,null,null,null,null,0.702031,1.040772,"""2015-12-20 00:00:26.571000""",null,0.301567,null,null,null,1436.760648,1671.20566,1279.975398,2160.482837
"""std""",5.982663,null,null,null,null,null,null,null,null,474503.072275,null,4.741281,null,null,null,null,null,null,null,null,null,3.692474,null,null,null,null,null,null,1194.145132,793.815448,541.86681,1499.188461
"""min""",0.01,null,null,null,null,null,"""-1""","""26 AGUSTOS""","""'44S""",196.0,"""079A_26004878070""",2007.0,"""200701""",null,null,null,null,null,null,null,0.0,0.0,"""2007-01-01""","""2M Alliance""",0.0,"""2M Alliance""","""ally""",null,5.93014,400.0,310.0,0.0
"""25%""",2.0,null,null,null,null,null,null,null,null,9.218686e6,null,2012.0,null,null,null,null,null,null,null,null,null,0.0,"""2012-02-27""",null,null,null,null,null,425.931559,1100.0,880.0,905.147059
"""50%""",2.533158,null,null,null,null,null,null,null,null,9.315202e6,null,2016.0,null,null,null,null,null,null,null,null,null,0.0,"""2016-02-20""",null,null,null,null,null,1261.373246,1590.0,1230.0,2036.911765
"""75%""",2.533158,null,null,null,null,null,null,null,null,9.430868e6,null,2019.0,null,null,null,null,null,null,null,null,null,2.0,"""2019-12-13""",null,null,null,null,null,2377.521062,2020.0,1530.0,3253.161765
"""max""",3729.25,null,null,null,null,null,"""ddedo""","""ZUMA""","""|SAL5""",9.979125e6,"""zzzz_ZZZZ""",2023.0,"""202312""",null,null,null,null,null,null,null,1.0,1123.25,"""2023-12-31""","""The Alliance""",1.0,"""The Alliance""","""non-ally""",null,8769.910108,15900.0,12070.0,17889.705882
